# Analysis of Fraud at UW and claims

## 1. Initialisation Phase


In [1]:
# variables indiquant si on relit ou non toute la base

# generation_source = 'Aggregator' 'Portal'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime 
import math
from plotly import graph_objects as go
import json 
import seaborn as sns
import pandas_profiling
import sys


C:\Users\guill\AppData\Local\Temp\ipykernel_41084\52057984.py:9: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  import pandas_profiling


In [3]:
## Python directory
### <--------------------------- TO BE CHANGED ---------------------------> ### 
PYTHON_DIRECTORY = 'https://github.com/ornikar/pricing-insurance/Shared Notebooks'
PYTHON_DIRECTORY = 'C:/Users/guill/GitHub/2023 Ornikar Pricing/pricing-insurance/Python Official Library/'
 
LATEX_MAIN_DIRECTORY = 'C:/Users/guill/GitHub/2023 Ornikar Pricing/pricing-insurance/Latex/OrnikarInsuranceAutomaticResults/'
LATEX_DIRECTORY = 'C:/Users/guill/GitHub/2023 Ornikar Pricing/pricing-insurance/Latex/OrnikarInsuranceAutomaticResults/Dashboard_Latex/'

RESULTS_DIRECTORY = 'C:/Users/guill/GitHub/2023 Ornikar Pricing/pricing-insurance/Results/'
### <--------------------------- TO BE CHANGED ---------------------------> ### 


sys.path.insert(1, PYTHON_DIRECTORY)


from importlib import reload  # Python 3.4+
import OrnikarDataFormat
import TransformedDataReading
import LinkQuotations
import LeFuretLibrary
import GraphicStatLibrary

# instruction pour forcer la relecture d'une base (utile quand on code ;) 
# LeFuretLibrary = reload(LeFuretLibrary)
# GraphicStatLibrary = reload(GraphicStatLibrary)


In [4]:
# Définition du mode de lecture / écriture des base : 
ModeGlobal = True # True or False : True pour tout lire
ModeSpeed = 'Full' # 'Full' means to keep all variables at the expense of Speed. Put 'Quick' instead
periodLeFuret =  '2023-03' # mois le Furet à lire
if ModeGlobal and ModeSpeed == 'Full':
    print("Full reading of BigQuery and all DataBases")
    mode = 'Google' 
    SizeVariablesKeptGW = 'Full' # We keep all the variables of GW except var_to_suppress.
elif ModeGlobal:
    print("Fast Mode through limited selection of variables - good for Clementine")
    mode = 'Google' 
    SizeVariablesKeptGW = 'Quick' # We keep all the variables of GW except var_to_suppress.
else: 
    mode = 'ReadLocalBase'
    SizeVariablesKeptGW = 'Full'


Full reading of BigQuery and all DataBases


In [5]:

# Import the data base

### <--------------------------- TO BE CHANGED ---------------------------> ### 
BASE_PATH = 'C:/Users/guill/GitHub/2023 Ornikar Empreinte/Data' 
BASE_PATH_LEFURET = 'C:/Users/guill/GitHub/2023 Coherence Tarifaire/Data' 
BASE_PATH_RES = RESULTS_DIRECTORY
### <--------------------------- TO BE CHANGED ---------------------------> ### 


In [6]:
## Variables to Keep Guidewire

# Liste Clémentine
var_process = ['account_id',  'account_number', 'submission_created_at', 'bind_date',  'job_id',  'job_number', 'policy_id',  'policy_number', 'tariff_viewed', 'wakam_called', 'tariff_accepted', 'wakam_pricing_version',
                'generation_source', 'online_offline' ,'ph_email']
var_risk = ['ph_email', 'loss_history_type', 'recommended_package', 'package_subtype', 'md_age', 'md_license_duration', 'vh_age',  'md_prefix',  'md_crm_coefficient', 'md_license_country', 'vh_sra_code', 'vh_license_plate','vh_annual_km','PACKAGE', ]  
# if SizeVariablesKeptGW = 'Full' alors à ces variables sont ajoutées TOUTES les variables sauf var_to_suppress
var_to_suppress = ['vh_sra_code','Unnamed: 0', 'insurer_name','submission_creation_user_name','update_user_id','ph_first_name','ph_last_name','ph_email','ph_phone'
] # vh_sra_code est en doublon

# Liste Guillaume 
var_process = ['account_id',  'account_number', 'submission_created_at', 'bind_date',  'job_id',  'job_number', 'policy_id',  'policy_number', 'tariff_viewed', 'wakam_called', 'tariff_accepted', 'policy_status', 'wakam_pricing_version',
                'generation_source', 'online_offline','ph_email' ]
var_risk = [ 'loss_history_type', 'recommended_package', 'package_subtype', 'md_age', 'md_license_duration', 'vh_age',  'md_prefix',  'md_crm_coefficient', 'md_license_country', 'vh_sra_code', 'vh_license_plate','vh_annual_km','PACKAGE', ]  

# suppression 'ph_gender' et remplacement par md_prefix

var_premium = ['total_premium_rpt','total_cost_rpt']


In [7]:
# Variable to Keep LeFuret

columnstoKeep = ['IdClient', 'SRA_Groupe', 'SRA_Classe', 'VEH_Antecedent',
       'premium_TTC', 'premium_HT', 'premium_RCC_TTC', 'premium_RCM_TTC',
       'premium_DRX_TTC', 'premium_PROPIL_TTC', 'premium_BDG_TTC',
       'premium_VLX_TTC', 'premium_INC_TTC', 'premium_CATNAT_TTC',
       'premium_CATECH_TTC', 'premium_EVCLM_TTC', 'premium_DOM_TTC', 'PP_RC',
       'PP_VOL', 'PP_INC', 'PP_BDG', 'PP_DOM', 'PP_SDC','message','ph_email']



columnstoKeepSmall = [ 'IdClient',
       'premium_TTC', 'premium_HT', 'premium_RCC_TTC', 'premium_RCM_TTC',
       'premium_DRX_TTC', 'premium_PROPIL_TTC', 'premium_BDG_TTC',
       'premium_VLX_TTC', 'premium_INC_TTC', 'premium_CATNAT_TTC',
       'premium_CATECH_TTC', 'premium_EVCLM_TTC', 'premium_DOM_TTC','message']

columnstoKeepSmall =['IdClient','premium_TTC','premium_HT','PP_RC','PP_VOL', 'PP_INC', 'PP_BDG', 'PP_DOM', 'PP_SDC','message'] 
# La prime Furet est TTC, premium HT est conservée pour le S/C

columnsPremium = ['premium_TTC', 'premium_HT', 'premium_RCC_TTC', 'premium_RCM_TTC',
       'premium_DRX_TTC', 'premium_PROPIL_TTC', 'premium_BDG_TTC',
       'premium_VLX_TTC', 'premium_INC_TTC', 'premium_CATNAT_TTC',
       'premium_CATECH_TTC', 'premium_EVCLM_TTC', 'premium_DOM_TTC','message']
columnsPremiumRC = [c+'_RC' for c in columnsPremium]
columnsPremiumRCP = [c+'_RCP' for c in columnsPremium]
columnsPremiumTR = [c+'_TR' for c in columnsPremium]

In [8]:
# Base le Furet méthode 2 : description automatique et suppression des variables non pertinentes.
# voir méthode que l'on retient au final

# Attention, clémentine, pour toi, faire un filtre sur [['custom_qr_id', 'couverture_demandée', 'mediane', 'annee_naissance', 'segment']]
featureLF_a_conserver = [['custom_qr_id', 'couverture_demandée', 'mediane', 'segment','md_age']] 

features_a_supprimer =['custom_qr_id','Unnamed: 0','Unnamed: 0.1','assureur','id','mediane', 'moyenne', 'position_vue', 'prix', 'id_externe_tarif', 'couverture','custom_offer_id',
                       'DTA_Incluse',	'DTA_Franchise_Montant',	'VI_Incluse',	'VI_Franchise_Montant',	'BDG_Incluse',	'BDG_Franchise_Montant',	'ASS_Panne_Incluse',	'ASS_Panne_Franchise_Montant',	'ASS_Accident_Incluse',	'ASS_Accident_Franchise_Montant',	'VR_Accident_Incluse',	'GC_Incluse',
                     	'custom_offer_id','PositionMediane','Position5HorsMediane','position_mer','devis','rappel','souscription']  
# un certain nombre de variables buguent car plusieurs formats. On les enlève dans un premier temps
# ! TODO il faudra essayer de les reintégrer
features_trop_large = ['code_postal','code_postal_travail','auto_sec_his_assure_actuel',
                       'auto_veh_tra_france','auto_pri_inf_conjoint_permis','auto_pri_inf_conjoint_autre_veh','auto_pri_his_sin_1_tiers','auto_pri_his_sin_2_tiers',
                       'auto_pri_his_sin_3_tiers','auto_pri_his_sin_4_tiers','auto_pri_his_sin_5_tiers',
                       'auto_pri_his_sin_corp_1_enquete','auto_pri_his_sin_corp_2_enquete','auto_pri_his_sin_corp_3_enquete','auto_pri_his_sin_corp_4_enquete','auto_pri_his_sin_corp_5_enquete',' ',
                       'auto_pri_his_sin_corp_1_prison', 'auto_pri_his_sin_corp_1_prison', 'auto_pri_his_sin_corp_2_prison','auto_pri_his_sin_corp_3_prison','auto_pri_his_sin_corp_4_prison','auto_pri_his_sin_corp_5_prison',
                       'auto_ass_dtl_souscription']
# auto_veh_garage_nt_departement_value était avant enlevé


In [9]:
### Variable Wakam
# Faire le Tri
VariableSRAWakam= ['SRA_CodeEnergie	SRA_PtacENkg','SRA_PuissanceReelleMaxiCEE',	'VEH_RatioWeightPower',	'SRA_Suspension	SRA_Alimentation',	'SRA_VitesseMaxi',	'SRA_TypeFreins',	'SRA_Carrosserie',	'VEH_Carrosserie_FOCUS',	'SRA_Transmission',	'SRA_NombrePlaces',	'SRA_ClasseReparationActuelle']
VariableSRAWakam= ['SRA_PuissanceReelleMaxiCEE','VEH_RatioWeightPower']



In [10]:
#Table de référence pour les libellés correspondant à une couverture donnée
var = {
    "libelle_import": ["RC", "RC+", "TR"], 
    "sexy_title": ["1. Tiers", "2. Tiers Etendue", "3. Tous Risques"],
    "2023-02" : ['All_Quote_Priced_02_2023_RC','All_Quote_Priced_02_2023_RC+','All_Quote_Priced_02_2023_TR'],
    "2023-03" : ['All_Quotes_RC_03_2023','All_Quotes_RC+_03_2023','All_Quotes_TR_03_2023'],
    }

couverture = ["tiers", "vol_incendie", "tous_risques"]
dico_couv = pd.DataFrame(var, index = couverture)
dico_couv

,libelle_import,sexy_title,2023-02,2023-03
tiers,RC,1. Tiers,All_Quote_Priced_02_2023_RC,All_Quotes_RC_03_2023
vol_incendie,RC+,2. Tiers Etendue,All_Quote_Priced_02_2023_RC+,All_Quotes_RC+_03_2023
tous_risques,TR,3. Tous Risques,All_Quote_Priced_02_2023_TR,All_Quotes_TR_03_2023


## 2. Reading DataBases

In [11]:
## Read either data base located on the computer (mode = 'ReadLocalBase') or through Google Big Query directly with the update of local data base (option = "Google")

df, df_previous_claims, df_previous_insurers, df_previous_terminations, df_previous_license_cancellations_suspensions  = TransformedDataReading.GoogleAllDataBaseReading(mode=mode, BASE_PATH =BASE_PATH)
#df, df_previous_claims, df_previous_insurers, df_previous_terminations, df_previous_license_cancellations_suspensions  = TransformedDataReading.GoogleAllDataBaseReading(mode='Google', BASE_PATH =BASE_PATH)

c:\Users\guill\anaconda3\envs\OnlyPIP\lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


c:\Users\guill\anaconda3\envs\OnlyPIP\lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
c:\Users\guill\anaconda3\envs\OnlyPIP\lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
c:\Users\guill\anaconda3\envs\OnlyPIP\lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has 

In [12]:
# Formatting Data 
OrnikarDataFormat = reload(OrnikarDataFormat)
dfFormat = OrnikarDataFormat.FormatGuideWire(df)
print('Number of policies UW : '+str(dfFormat['tariff_accepted'].sum()))


['submission_created_at', 'job_close_date', 'bind_date', 'issue_date', 'cancellation_date', 'original_effective_date', 'period_start', 'period_end', 'creation_date', 'update_date', 'price_computed_at', 'written_date', 'policy_period_start', 'policy_period_end']
Number of policies UW : 2464


In [13]:
dfFormat["md_license_duration"]

0          4.0
1          1.0
2          0.0
3         33.0
4          2.0
          ... 
235972    18.0
235973    18.0
235974    18.0
235975    18.0
235976    18.0
Name: md_license_duration, Length: 235977, dtype: float64

In [14]:
# Definition de var_other qui contient toutes les variables qui n'ont pas été explicitement rejetées dans var_suppress
from itertools import chain
var_other = [col for col in dfFormat.columns if col not in list(chain(var_process, var_risk, var_premium ,var_to_suppress,['account_id', 'submission_created_at']))  ]
for col in var_other:
    if col == 'vh_sra_code':print('probleme')
    if col == 'account_id':print('problème')
    if col == 'md_license_country':print(col)
print('liste var_others')
print(var_other)

liste var_others
['policy_period_public_id', 'policy_public_id', 'sf_external_contact_id', 'policy_term_public_id', 'wakam_quote_id', 'wakam_price_test', 'wakam_price_test_perc_applied', 'job_type', 'job_public_id', 'job_close_date', 'issue_date', 'cancellation_date', 'cancellation_nature', 'sf_submission_id', 'sf_opportunity_id', 'original_effective_date', 'period_start', 'period_end', 'period_id', 'creation_date', 'update_date', 'price_computed_at', 'last_policy_period_id', 'product_code', 'policy_term_id', 'risk_type', 'written_date', 'submission_creation_user_id', 'submission_creation_user_email', 'is_from_aggregator', 'is_anonymous', 'session_id', 'utm_source', 'utm_medium', 'utm_campaign', 'utm_term', 'utm_content', 'policy_risk_profile', 'policy_risk_profile_edu', 'has_formulas', 'has_selected_formula', 'account_public_id', 'ph_prefix', 'ph_birthdate', 'md_property_owner_type', 'md_accommodation_type', 'phc_policy_contact_role_public_id', 'phc_sf_external_contact_id', 'md_driver

In [15]:
# Suppression des doublons et conservation uniquement des variables d'intérêt
''' sortOrder = [True,True] : choose the first quote of a specific account; [True,False] : the last one ''' 
TransformedDataReading = reload(TransformedDataReading)

subf=TransformedDataReading.UniqueRow(dfFormat,sortOrder = [True, True], var_process = var_process,
            var_risk = var_risk ,       
            var_premium = var_premium,var_other = var_other, size = SizeVariablesKeptGW)
 

Suppression of Doublons 
Il existe 235977 lignes avant dédoublonnage
Il existe 209054 lignes lorsqu'on dédoublonne par account_id


In [16]:
# Base SRA
df_SRA = pd.read_csv(f'{BASE_PATH}/Wakam_vehiculier_auto 04-2023 (dwld PlugPlay).csv', sep=';',low_memory=False)
# On récupère les 3 variables SRA manquantes (classe, poids véhicule vide, puissance DIN)
df_SRA = df_SRA[['SRAIdentifiant','Classe','PoidsASec','PuissanceReelleMaxiDINenCv']]
# Ajout de variables comme le vehicule Ratio
df_SRA = OrnikarDataFormat.FormatSRA(df_SRA,Stat=False)

In [17]:
#pandas_profiling.ProfileReport(subf, minimal=True)

In [18]:
# Merge of all the Databases of GuideWire into a unique One
OrnikarDataFormat = reload(OrnikarDataFormat)
#display(df,df_SRA,df_previous_claims,df_previous_insurers,df_previous_terminations,df_previous_license_cancellations_suspensions) 
BaseQuotation = OrnikarDataFormat.mergeBaseGuideWire(dfFormat,df_SRA,df_previous_claims,df_previous_insurers,df_previous_terminations,df_previous_license_cancellations_suspensions,Stat=True)
print('Number of policies UW : '+str(BaseQuotation['tariff_accepted'].sum()))
#pandas_profiling.ProfileReport(BaseQuotation, minimal=True)
BaseQuotation = OrnikarDataFormat.FormatGuideWire(BaseQuotation)
print('Number of policies UW : '+str(BaseQuotation['tariff_accepted'].sum()))

segment marketing


,segment
3. Standard,134679
2. Jeune permis - > 21 ans,39026
1. Jeune permis - 18-21 ans,28278
4. Sans Antécédent,25515
5. Aggravé (CRM > 100),7979
<NA>,499


segment Reinsurer


,ReinsurerSegment
3. Standard,134679
2. Young Driver with No EduScore,50280
4. Interrupted,25515
1b. Young Driver with EduScore (bad),9254
1a. Young Driver with EduScore (Good),8268
3b. Standard - poor CRM,4642
2b.Young Driver - poor CRM,2896
4b. Interrupted - poor CRM,441
<NA>,1


case with NA Segment 499


,segment,md_risk_profile,md_crm_coefficient,md_license_duration,duree_assurance_sur_3ans,md_edu_score,md_age
1150,<NA>,Young Driver,1.0,0.0,0.000000,<NA>,17.0
3244,<NA>,Unknown,1.0,0.0,8.969383,<NA>,17.0
3245,<NA>,Unknown,1.0,0.0,8.969383,<NA>,17.0
3272,<NA>,Unknown,1.0,0.0,0.000000,<NA>,17.0
3274,<NA>,Unknown,1.0,0.0,0.000000,<NA>,17.0
...,...,...,...,...,...,...,...
224684,<NA>,Unknown,1.0,0.0,35.976098,<NA>,17.0
228239,<NA>,Young Driver,1.0,0.0,8.739399,<NA>,17.0
232400,<NA>,Young Driver,1.0,0.0,8.969383,<NA>,17.0
234199,<NA>,Unknown,1.0,0.0,11.992033,<NA>,17.0


case with NA ReinsurerSegment


,ReinsurerSegment,md_risk_profile,md_crm_coefficient,md_license_duration,duree_assurance_sur_3ans,md_edu_score
53341,<NA>,Unknown,NaN,9.0,35.976098,<NA>


Output Database with 235976 lines and 208 Columns
Number of policies UW : 2464
['cancellation_date', 'submission_created_at_Timestamp']
Number of policies UW : 2464


In [19]:
# Indicatrices - Règles GW 
OrnikarDataFormat = reload(OrnikarDataFormat)
BaseQuotation = OrnikarDataFormat.IndicatorsUnderwritingRules(df_temp=BaseQuotation,Stat = False)
print('Number of policies UW : '+str(BaseQuotation['tariff_accepted'].sum()))


Number of policies UW : 2464
